# 9.1. Putting a Wrapper Around a Function

In [3]:
# define a decorate function 
import time
from functools import wraps 

def timethis(func):
    '''
    Decorator that reports the execution time
    '''
    @wraps(func)
    # the use of *args and **kwargs is there to make sure that any input arguments can be accepted.
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result 
    return wrapper

# Here is an example of using the decorator 
@timethis
def countdown(n):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1 
countdown(100000)

countdown 0.007312774658203125


# 9.2. Preserving Function Metadata When Writing Decorators

In [8]:
import time 
from functools import wraps 

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result 
    return wrapper 

@timethis
def countdown(n:int):
    '''
    Counts down 
    '''
    while n > 0:
        n -= 1

countdown(100000)
countdown.__name__
countdown.__doc__
countdown.__annotations__

countdown 0.006569862365722656


{}

# 9.3. Unwrapping a Decorator

In [14]:
import time 
from functools import wraps 

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result 
    return wrapper 

@timethis
def countdown(n:int):
    '''
    Counts down 
    '''
    while n > 0:
        n -= 1

#countdown(10000)
countdown.__wrapped__(100)

# Gaining direct access to the unwrapped function behind a decorator can be useful for debugging, introspection,and other operations involving functions.

from functools import wraps 

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper 

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

@decorator1
@decorator2
def add(x, y):
    return x + y
add(2,3)
add.__wrapped__.__wrapped__(2,3)

Decorator 1
Decorator 2


5

# 9.4 Defining a Decorator That Takes Arguments

In [ ]:
from functools import wraps
import logging 

def logged(level, name=None, message=None):
    '''
    Add logging to a function, level is the logging
    level, name is the logger name, and message is the 
    log message, if name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

# 9.5. Definiing a Decorator with User Adjustable Attributes

In [17]:
from functools import wraps, partial 
import logging 

# Utility decorator to attach a function as an attribute of obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func 

def logged(level, name=None, message=None):
    '''
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the 
    log message. If name and message aren't specified, they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        
        # Attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level 
            level = newlevel 
        
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg 
            logmsg = newmsg 
        
        return wrapper
    return decorate 

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y 

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

import logging
logging.basicConfig(level=logging.DEBUG)
add(2,3)

# Change the log message 
add.set_message('Add called')
add(2, 3)

# Change the log level 
add.set_level(logging.WARNING)
add(2,3)

DEBUG:__main__:add
DEBUG:__main__:Add called


5

# 9.6.Defining a Decorator That Takes an Optional Argument

In [ ]:
from functools import wraps, partial 
import logging 

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)

    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__ 
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*agrs, **kwargs)
    return wrapper

# Exmaple me
@logged
def add(x, y):
    return x + y

@logged(level=logging.CRITICAL, name='example')
def spam():
    print('Spam!')

# 9.7. Enforcing Type Checking on a Function Using a Decorator

In [19]:
# enforcing type contracts on the input arguments to a function
from inspect import signature
from functools import wraps 

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking 
        if not __debug__:
            return func 
        
        # Map function argument names to supplied types
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                            'Argument {} must be {}'.format(name, bound_types[name])
                        )
            return func(*args, **kwargs)
        return wrapper
    return decorate

@typeassert(int, int)
def add(x, y):
    return x + y 

add(2, 3)
add(2, 'hello')

TypeError: Argument y must be <class 'int'>